# Model Quantization for Communication
In the previous examples, we used numpy in float32 for communication. To reduce the message size, we can use model precision conversion and quantization 
from float32 to 16-bit, 8-bit, and 4-bit for communication. Quantization is enabled by NVFlare's [filter mechanism](https://nvflare.readthedocs.io/en/main/programming_guide/filters.html). We can use the following command to run the federated training with model quantization.
16-bit is a direct precision conversion, while 8-bit, 4-bit quantization is performed by [bitsandbytes](https://github.com/bitsandbytes-foundation/bitsandbytes/tree/main).
Note that 4-bit quantizations (`fp4` or `nf4`) need device support.

## Data Preparation
Again, we use one dataset to illustrate the SFT. We download and preprocess [databricks-dolly-15k](https://huggingface.co/datasets/databricks/databricks-dolly-15k).

In [ ]:
! git clone https://huggingface.co/datasets/databricks/databricks-dolly-15k /tmp/nvflare/dataset/llm/dolly
! python utils/preprocess_dolly.py --training_file /tmp/nvflare/dataset/llm/dolly/databricks-dolly-15k.jsonl --output_dir /tmp/nvflare/dataset/llm/dolly

We run the same SFT pipeline with different quantization configurations:

In [ ]:
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_16 --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_16 --train_mode SFT --quantize_mode float16
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_8 --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_8 --train_mode SFT --quantize_mode blockwise8
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_fp4 --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_fp4 --train_mode SFT --quantize_mode float4
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_nf4 --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_nf4 --train_mode SFT --quantize_mode normfloat4

For message reduce, from float32 to 16-/8-/4-bit, the message size (in MB) of Llama-3.2-1B model are reduced to: 

| Quantization      | Raw Model Size | Quantized Model Size | Quantization Meta Size |
|-------------------|----------------|----------------------|------------------------|
| float16           | 5716.26        | 2858.13              | 0.00                   |
| blockwise8        | 5716.26        | 1429.06              | 1.54                   |
| float4            | 5716.26        | 714.53               | 89.33                  |
| normalized float4 | 5716.26        | 714.53               | 89.33                  |

Note that quantization will generate additional meta data, which can be significant for 4-bit cases.

## Model Communication with Tensor
In addition, since the model is trained with bf16, instead of first converting to numpy in float32, we can directly communicate with tensor in bf16 to avoid the message size inflation due to the conversion. 
We can use the following command to run the federated training with direct tensor communication, without and with quantization:

In [ ]:
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_tensor --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_tensor --train_mode SFT  --message_mode tensor
! python sft_job.py --client_ids dolly --data_path /tmp/nvflare/dataset/llm/ --workspace_dir /tmp/nvflare/workspace/llm/dolly_fl_sft_tensor_fp4 --job_dir /tmp/nvflare/workspace/jobs/llm_hf_sft_tensor_fp4 --train_mode SFT  --message_mode tensor --quantize_mode float4

In this case, since the tensor is in bf16, and the quantization reduces it to float4, the message size change is thus:
```
Before quantization: 2858.13 MB. After quantization: 714.53 MB with meta: 89.33 MB.
```

## Training Curves

The SFT curves are shown below, we can see it achieves decent alignments. These results show that for the example training schemes and data, model precision conversion / quantization does not significantly impact the training while reducing the message size to 1/2, 1/4, and even 1/8, which can significantly reduce the message size, making it crucial for transmitting LLM updates.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /tmp/nvflare/workspace/llm

Quantization significantly reduced the communication burden by reducinng the message size sent over the network, however at local level, memory usage is still demanding to prepare the messages - large memory needs to be allocated to hold the LLM weights. Therefore, let's move on to the next section addressing this challenge - [LLM Streaming](../08.5_llm_streaming/LLM_streaming.ipynb))